In [1]:
%load_ext autoreload
%autoreload 2

import os


import sys

import scipy
import scipy.io
import time

import os.path
import warnings
warnings.filterwarnings('ignore')
os.environ["KMP_WARNINGS"] = "FALSE"


import tensorflow as tf
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

CHECKPOINT_PATH = "/home/vlq26735/code/PhD/GradientPathologiesPINNs/checkpoints/1DPoisson/"
MODE = "M3"

src_path = os.path.abspath(os.path.join('../../'))
print(f"src_path= {src_path}")

if src_path not in sys.path:
    sys.path.insert(0, src_path)


from src.cases.Poisson import M3



src_path= /home/vlq26735/code/PhD/GradientPathologiesPINNs


In [2]:
class Sampler:
    # Initialize the class
    def __init__(self, dim, coords, func, name=None):
        self.dim = dim
        self.coords = coords
        self.func = func
        self.name = name

    def sample(self, N):
        x = self.coords[0:1, :] + (self.coords[1:2, :] - self.coords[0:1, :]) * np.random.rand(N, self.dim)
        y = self.func(x)
        return x, y

In [3]:
# Define solution and its Laplace
a = 4

def u(x, a):
  return np.sin(np.pi * a * x)

def u_xx(x, a):
  return -(np.pi * a)**2 * np.sin(np.pi * a * x)

In [4]:
# Define computional domain
bc1_coords = np.array([[0.0], [0.0]])
bc2_coords = np.array([[1.0], [1.0]])
dom_coords = np.array([[0.0], [1.0]])

# Training data on u(x) -- Dirichlet boundary conditions

nn  = 100

X_bc1 = dom_coords[0, 0] * np.ones((nn // 2, 1))
X_bc2 = dom_coords[1, 0] * np.ones((nn // 2, 1))
X_u = np.vstack([X_bc1, X_bc2])
Y_u = u(X_u, a)

X_r = np.linspace(dom_coords[0, 0], dom_coords[1, 0], nn)[:, None]
Y_r = u_xx(X_r, a)

nn = 1000
X_star = np.linspace(dom_coords[0, 0], dom_coords[1, 0], nn)[:, None]
u_star = u(X_star, a)
r_star = u_xx(X_star, a)

nIter =40001
bcbatch_size = 500
ubatch_size = 5000
mbbatch_size = 128



# Define model
mode = 'M3'
layers = [1, 500,500, 1]



iterations = 1
methods = [ "mini_batch"]

result_dict =  dict((mtd, []) for mtd in methods)


# Create residual sampler
gpu_options = tf.GPUOptions(visible_device_list="0")
tf.reset_default_graph()
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,allow_soft_placement=False, log_device_placement=False))
# with  as sess:

model = M3.Possion_M3(layers, X_u, Y_u, X_r, Y_r , mode , CHECKPOINT_PATH , sess)    

# Train model
start_time = time.time()

print("mini_batch method is used")
model.trainmb(nIter, mbbatch_size)
elapsed = time.time() - start_time

# Predictions
u_pred = model.predict_u(X_star)
r_pred = model.predict_r(X_star)
# Predictions

error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
error_r = np.linalg.norm(r_star - r_pred, 2) / np.linalg.norm(r_star, 2)

model.print('elapsed: {:.2e}'.format(elapsed))

model.print('Relative L2 error_u: {:.2e}'.format(error_u))
model.print('Relative L2 error_r: {:.2e}'.format(error_r))


model.print('mean value of lambda_bc: {:.2e}'.format(np.average(model.adaptive_constant_bcs_log)))
model.print('first value of lambda_bc: {:.2e}'.format(model.adaptive_constant_bcs_log[0]))
model.print('Relative L2 error_u: {:.2e}'.format(error_u))
model.print('Relative L2 error_v: {:.2e}'.format(error_r))
model.print("file directory:" , model.dirname)
model.save_NN()
model.plot_grad()
model.plot_lambda()
model.plt_prediction( X_star , u_star , u_pred)
# sess.close()  
    

2024-02-27 16:37:53.602215: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-27 16:37:53.675023: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2445400000 Hz
2024-02-27 16:37:53.675330: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55eb23203ff0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-02-27 16:37:53.675355: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-02-27 16:37:53.675538: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.







mini_batch method is used


It: 0, Loss: 9.516e+00, Loss_bcs: 1.386e-01, Loss_res: 9.377e+00 ,Time: 14.75
mean_grad_bcs: 9.8384e-02
mean_grad_res: 1.1551e-02
self.lam_bc: 1.1174e+00
self.lam_res : 9.5174e+00
Gradients information stored ...
It: 100, Loss: 9.140e+00, Loss_bcs: 1.286e-04, Loss_res: 9.140e+00 ,Time: 0.13
Gradients information stored ...
It: 200, Loss: 8.800e+00, Loss_bcs: 2.463e-04, Loss_res: 8.799e+00 ,Time: 0.13
It: 300, Loss: 8.371e+00, Loss_bcs: 7.828e-04, Loss_res: 8.371e+00 ,Time: 0.03
It: 400, Loss: 7.916e+00, Loss_bcs: 2.154e-03, Loss_res: 7.913e+00 ,Time: 0.07
It: 500, Loss: 7.426e+00, Loss_bcs: 4.606e-03, Loss_res: 7.421e+00 ,Time: 0.04
It: 600, Loss: 6.686e+00, Loss_bcs: 1.012e-02, Loss_res: 6.676e+00 ,Time: 0.03
It: 700, Loss: 5.119e+00, Loss_bcs: 2.442e-02, Loss_res: 5.095e+00 ,Time: 0.02
It: 800, Loss: 3.380e+00, Loss_bcs: 8.888e-03, Loss_res: 3.371e+00 ,Time: 0.02
It: 900, Loss: 2.426e+00, Loss_bcs: 1.820e-02, Loss_res: 2.408e+00 ,Time: 0.03
It: 1000, Loss: 1.526e+00, Loss_bcs: 2.114e

KeyboardInterrupt: 

In [5]:
elapsed = time.time() - start_time

# Predictions
u_pred = model.predict_u(X_star)
r_pred = model.predict_r(X_star)
# Predictions

error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
error_r = np.linalg.norm(r_star - r_pred, 2) / np.linalg.norm(r_star, 2)

model.print('elapsed: {:.2e}'.format(elapsed))

model.print('Relative L2 error_u: {:.2e}'.format(error_u))
model.print('Relative L2 error_r: {:.2e}'.format(error_r))


model.print('mean value of lambda_bc: {:.2e}'.format(np.average(model.adaptive_constant_bcs_log)))
model.print('first value of lambda_bc: {:.2e}'.format(model.adaptive_constant_bcs_log[0]))
model.print('Relative L2 error_u: {:.2e}'.format(error_u))
model.print('Relative L2 error_v: {:.2e}'.format(error_r))

model.save_NN()
model.plot_grad()
model.plot_lambda()
model.plt_prediction( X_star , u_star , u_pred)

elapsed: 4.55e+02
Relative L2 error_u: 3.28e-03
Relative L2 error_r: 7.77e-04
mean value of lambda_bc: 1.12e+00
first value of lambda_bc: 1.12e+00
Relative L2 error_u: 3.28e-03
Relative L2 error_v: 7.77e-04
Save uv NN parameters successfully in %s .../home/vlq26735/code/PhD/GradientPathologiesPINNs/checkpoints/1DPoisson/Feb-27-2024_16-37-53-676244_M3
Final loss total loss: 1.008243e-02
Final loss loss_res: 1.006646e-02
Final loss loss_bcs: 1.597392e-05


In [ ]:
Relative L2 error_u: 5.63e-02
Relative L2 error_v: 4.55e-02